## Part 1: Questions to text and lectures.

## Part 2: Random forest and weather

#### Part 2A: Random forest binary classification. 
* Using the and instructions and material from Week 7, build a *random forest* classifier to distinguish between two types (you choose) of crime using on spatio-temporal (where/when) features of data describing the two crimes. When you're done, you should be able to give the classifier a place and a time, and it should tell you which of the two  types of crime happened there.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import seaborn as sns
import datetime
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy import stats

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from matplotlib import cm



In [12]:
DATASET_LOCATION = '/Users/yuchao/Desktop/courses/datavisualization/crime.csv'
PICKLE_PATH = 'pickles'

In [13]:
def pickled(pickle_name):
    def inner_function(func):
        def wrapper(*args, **kwargs):
            path = os.path.join(PICKLE_PATH, pickle_name)
            if os.path.exists(path):
                with open(path, "rb") as f:
                    result = pickle.load(f)
            else:
                result = func(*args, **kwargs)
                with open(path, "ab") as f:
                    pickle.dump(result, f)
            return result
        return wrapper
    return inner_function

In [14]:
@pickled("ex2_data")
def get_data():
     return pd.read_csv(DATASET_LOCATION)

In [15]:
def coerce_to_datetime(df):
    
    df['Date'] = pd.to_datetime(df['Date'])
#     df['newtime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['Time'] =pd.to_datetime(df['Time'], format='%H:%M')
    return df

In [16]:
df = get_data()
df = coerce_to_datetime(df)
df = df[df.Y != 90]

In [17]:
weekdays_multipliers = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
}

def get_dataframe_new(categories,df):
    
    df_empty =pd.DataFrame()
    i = 0
    for category in categories:
        
        dm = df[df.Category == category]
        dm=dm.copy()
        dm['Date'] +=  pd.to_timedelta(dm['Time'].dt.hour, unit='h')
        hourofweek = [weekdays_multipliers[i]*24  for i in dm["DayOfWeek"].values] + dm['Time'].dt.hour.values
        district = dm.PdDistrict
        year =dm['Date'].dt.year.values
        month = dm['Date'].dt.month.values
        dayofweek = [weekdays_multipliers[i] for i in dm["DayOfWeek"].values]
        date = dm['Date']
        data = {'hourofweek':hourofweek,'district':district,'year':year,'month':month,'date':date,'dayofweek':dayofweek}
        df_new= pd.DataFrame(data, columns = ['hourofweek','district','year','month','date','dayofweek'])
        df_new['category'] = category
        
        df_empty = df_empty.append(df_new)
        i +=1
        df_empty.index = range(len(df_empty))
        
    df_empty['category'] = df_empty['category'].astype('category')
    df_empty['category'] = df_empty['category'].cat.codes
    df_empty['district'] = df_empty['district'].astype('category')
#     df_empty['district'] = df_empty['district'].cat.codes
    
    
    return df_empty

df_tree = get_dataframe_new(['BURGLARY','FRAUD'],df)

In [18]:
num_burglary = df_tree[df_tree.category == 0].shape[0]
num_fraud = df_tree[df_tree.category == 1].shape[0]
print(num_burglary,num_fraud)

91542 41540


In [19]:
def kfold_crossvaliadation(n,feature_col,column,df):
    X = df[feature_col]
    Y = df[column]
    
    clf=RandomForestClassifier(n_estimators=100,random_state=0,max_depth=16,min_samples_split=2)
    
    accuracy = cross_val_score(clf, X, Y, cv=n)
    
    return accuracy

In [20]:
col = ['hourofweek','month','district','dayofweek','year']

def feature_importance(df,column,col):
    x = df[column]
    y = df[col]
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)
    clf=RandomForestClassifier(n_estimators=500,random_state=0,max_depth=None,min_samples_split=2)
    clf=clf.fit(x_train,y_train)
    y_train_pred = clf.predict(x_train)
    train_accuracy = metrics.accuracy_score(y_train_pred,y_train)
    y_pred = clf.predict(x_test)
    print(classification_report(y_test, y_pred))
    feature_imp = pd.Series(clf.feature_importances_,index=column).sort_values(ascending=False)
    return feature_imp,train_accuracy

df_tree['district'] = df_tree['district'].cat.codes
feature_importance(df_tree,col,'category')

              precision    recall  f1-score   support

           0       0.73      0.80      0.76     27536
           1       0.44      0.35      0.39     12389

    accuracy                           0.66     39925
   macro avg       0.59      0.57      0.58     39925
weighted avg       0.64      0.66      0.65     39925



(hourofweek    0.523470
 month         0.208719
 year          0.158272
 district      0.077081
 dayofweek     0.032457
 dtype: float64,
 0.9328230836115375)

In [21]:
#without one-hot encoding
fea_col = ['hourofweek','month','year','district']
src = kfold_crossvaliadation(5,fea_col,'category',df_tree)
print("Accuracy: %0.3f (+/- %0.3f)" % (src.mean(), src.std() * 2))

Accuracy: 0.685 (+/- 0.029)


In [22]:
#with one-hot encoding
df_decision_tree = get_dataframe_new(['BURGLARY','FRAUD'],df)
df_district = pd.get_dummies(df_decision_tree.district)
df_encoding = df_decision_tree.join(df_district)
col = ['hourofweek','month','year','SOUTHERN', 'TARAVAL', 'BAYVIEW', 'NORTHERN', 'INGLESIDE','RICHMOND', 'MISSION', 'CENTRAL', 'TENDERLOIN', 'PARK']
arc = kfold_crossvaliadation(5,col,'category',df_encoding)
print("Accuracy: %0.2f (+/- %0.2f)" % (arc.mean(), arc.std() * 2))

Accuracy: 0.68 (+/- 0.02)


- Explain about your choices for training/test data, features, and encoding.

> * The crime burglary and fraud were chosen to establish a classification model. These two crimes are imbalanced, the proportion is 2:1. Through several trials, balanced data decreased the accuracy in this model, so we didn't balance the data. However, balancing is a good way to increase the weight of the minority when one class has an extreme small size of data. In addition, it increases the recall rate. On the other hand, balancing are usually made by under sampling and over sampling.Under sampling decreases the size of dataset, which may lead to weeken the learning ability of the model. Over sampling may result in overfitting.

> * As for overfitting, the model works well on training data, but the precision is much lower, using testing data. Therefore, the model is a bit overfitting or the features are not suitable to be used for this model. We used both k-fold cross validation and hold-out crossification. 

> * Features' impotrance was tested to get the most relevant features so that the model's dimension could be decreased. The features 'hourofweek','month','year' and 'districts' were used in this model.

> * In general, one hot encoding provides better resolution of the data for the model and most models end up performing better. Nevetherless, it increases the depth of the tree and the complexity of the model here. The accuracy decreased with one-hot coding. Therefore, we didn't use one-hot encoding in this model.


- Report accuracy. Discuss the model performance.
> Using the k-fold cross validation. The accuracy is 68.5%. According to the classification report, this model fits well on identifying the crime 'BURGLARY' whose precison is 75%.  The precision,recall and f1 are very high for the crime 'BURGLARY. In addition, the model works well on training data but doesn't fit well for testing data so this model suffers from a bit overfitting and features selection. Also we found that using hold-out crossvalidation can decrease the accuracy of this model to 66%.

#### Part 2B: Info from weather features.
* Now add features from weather data to your random forest. 

In [23]:
df_weather = pd.read_csv('/Users/yuchao/Desktop/courses/datavisualization/weather_data.csv')
df_weather.dropna()
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_weather['date'] = df_weather['date'].dt.tz_convert(None)

In [24]:
df_crime_weather = df_tree.set_index('date').join(df_weather.set_index('date'))
df_crime_weather = df_crime_weather.dropna()
df_crime_weather['weather'] = df_crime_weather['weather'].astype('category')
df_crime_weather['weather'] = df_crime_weather['weather'].cat.codes
df_crime_weather

,hourofweek,district,year,month,dayofweek,category,temperature,humidity,weather,wind_speed,wind_direction,pressure
date,,,,,,,,,,,,
2012-10-01 13:00:00,13,1,2012,10,0,0,16.330000,88.0,10,2.0,150.0,1009.0
2012-10-01 15:00:00,15,1,2012,10,0,1,16.310618,86.0,20,2.0,141.0,1009.0
2012-10-01 17:00:00,17,6,2012,10,0,0,16.281869,84.0,20,2.0,129.0,1009.0
2012-10-01 18:00:00,18,7,2012,10,0,1,16.267494,83.0,20,2.0,123.0,1010.0
2012-10-01 18:00:00,18,5,2012,10,0,1,16.267494,83.0,20,2.0,123.0,1010.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-10-27 23:00:00,119,7,2017,10,4,0,27.910000,20.0,20,2.0,120.0,1013.0
2017-10-27 23:00:00,119,1,2017,10,4,0,27.910000,20.0,20,2.0,120.0,1013.0
2017-10-28 00:00:00,120,4,2017,10,5,0,28.920000,22.0,20,2.0,130.0,1013.0


In [25]:
weather_col = ['hourofweek','district','month','temperature','humidity','weather','wind_speed','wind_direction','pressure','year','dayofweek']
feature_importance(df_crime_weather,weather_col,'category')

              precision    recall  f1-score   support

           0       0.71      0.87      0.78      9138
           1       0.47      0.24      0.32      4290

    accuracy                           0.67     13428
   macro avg       0.59      0.56      0.55     13428
weighted avg       0.63      0.67      0.63     13428



(temperature       0.150812
 hourofweek        0.142138
 district          0.121779
 wind_direction    0.119841
 humidity          0.102643
 pressure          0.100349
 month             0.062794
 weather           0.062738
 wind_speed        0.058144
 year              0.040635
 dayofweek         0.038127
 dtype: float64,
 0.9766053874632963)

In [26]:
#with weather data
column1 = ['temperature','hourofweek','district','wind_direction','humidity']
scr2 = kfold_crossvaliadation(5,column1,'category',df_crime_weather)
print("Accuracy: %0.3f (+/- %0.3f)" % (scr2.mean(), scr2.std() * 2))

Accuracy: 0.677 (+/- 0.005)


In [27]:
#without weather data
column2 = ['hourofweek','month','year','district']
scr3 = kfold_crossvaliadation(5,column2,'category',df_crime_weather)
print("Accuracy: %0.3f (+/- %0.3f)" % (scr3.mean(), scr3.std() * 2))

Accuracy: 0.624 (+/- 0.081)


- Report accuracy. 
> Using k-fold cross validation, the accuracy is 67.7%.
- Discuss how the model performance changes relative to the version with no weather data.
> In order to control the sample size, we used the same dataset to build two models with weather data and without weather data. The accuracy increased 5% with weather data. The weather data does help in improving the performance of the model. 
- Discuss what you have learned about crime from including weather data in your model.
> Firstly, the accuracy increases with weather data which shows that the weather does influence the crimes. Secondly, from the feature importance, the weather of the day plays little importance in identifying the crimes. So it can be concluded that no matter how the weather condition is, both crimes have the eaqual possibility to take palce. Thirdly, temperature does help identifying the crimes. we can rougly know which crime is more likely to happen according to the temperature.

## Part 3: Data visualization

* Create the Bokeh visualization from Part 2 of the Week 8 Lecture, displayed in a beautiful `.gif` below. 
* Provide nice comments for your code. Don't just use the `# inline comments`, but the full Notebook markdown capabilities and explain what you're doing.

In [28]:
focuscrimes = ['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT']
focuscrimes.sort()

**First step**

- use a function to filter the data by date range '01-01-2010' - '12-31-2018'

In [29]:
def filter_by_date_range(df, start_date, end_date):
    df['Date'] = pd.to_datetime(df['Date'])
    mask = (df['Date'] > start_date) & (df['Date'] <= end_date)
    return df.loc[mask]
dm = filter_by_date_range(df,'01-01-2010','12-31-2018')
dm = dm.copy()
dm['Hour'] = dm['Time'].dt.hour

**Second step**

- group the data by Category and Hour
- reindex the new dataframe to ensure the value arranged in the order of 0-23 in the hour column

In [33]:
seq = [i for i in range(24)]

df_groupby_hour = dm.groupby('Category')['Hour'].value_counts()
df_groupby_hour = df_groupby_hour.reindex( pd.MultiIndex.from_product([df_groupby_hour.index.levels[0], seq], 
                                names=['district', 'hourofweek']),fill_value=0)
df_groupby_hour

district     hourofweek
ARSON        0             147
             1             135
             2             139
             3             144
             4             128
                          ... 
WEAPON LAWS  19            834
             20            672
             21            669
             22            776
             23            672
Name: Hour, Length: 936, dtype: int64

**Third step**

- use a function to calcuate the crime rate of each crime in each district
- build a new dataframe to store values for next step

In [31]:
def get_dataframe(df,categories):
    new_df =pd.DataFrame()
    new_df['Hour'] = [i for i in range(24)]
    
    for category in categories:
        
        total_crime = df[category].sum()
        new_df[category] = df[category]/total_crime
        
    return new_df

prep_data = get_dataframe(df_groupby_hour,focuscrimes)  
prep_data

,Hour,ASSAULT,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,LARCENY/THEFT,PROSTITUTION,ROBBERY,STOLEN PROPERTY,TRESPASS,VANDALISM,VEHICLE THEFT,WEAPON LAWS
0,0,0.055904,0.040359,0.051437,0.120853,0.034659,0.080072,0.039264,0.131556,0.054577,0.043398,0.028363,0.054167,0.036015,0.052582
1,1,0.047575,0.027917,0.038109,0.111671,0.020525,0.077091,0.025160,0.096475,0.058982,0.033172,0.020439,0.037969,0.024202,0.037881
2,2,0.043125,0.031573,0.032486,0.099230,0.016528,0.069938,0.015378,0.061565,0.059116,0.028976,0.025482,0.035327,0.018246,0.032514
3,3,0.023141,0.033317,0.017909,0.047393,0.012219,0.026624,0.009912,0.036285,0.036620,0.022945,0.021160,0.025657,0.012011,0.022558
4,4,0.014381,0.029745,0.014994,0.017180,0.009076,0.014107,0.006538,0.019948,0.022967,0.019405,0.016027,0.017641,0.010079,0.015401
5,5,0.012414,0.026130,0.059350,0.011256,0.005142,0.004967,0.006605,0.010146,0.019033,0.016258,0.038178,0.014498,0.010039,0.007934
6,6,0.015702,0.023285,0.118076,0.012737,0.010762,0.009140,0.009985,0.008255,0.017688,0.016782,0.073654,0.016927,0.015338,0.011123
7,7,0.022719,0.031863,0.101000,0.008590,0.025063,0.019869,0.014998,0.005159,0.016410,0.024256,0.072844,0.021527,0.022668,0.021235
8,8,0.033597,0.048107,0.074761,0.007998,0.033390,0.017286,0.025908,0.005159,0.019537,0.027796,0.064019,0.031517,0.033046,0.025591
9,9,0.036154,0.044679,0.052895,0.013626,0.038427,0.021458,0.031945,0.002923,0.021958,0.031336,0.056726,0.029998,0.033106,0.033370


**Fourth step**

- convert the dataframe to ColumnDataSource
- create a figure and adjust the parameters (width,height,label,title)
- create 14 vbars with different paramters and store their names and vbars in a list 
- create a lengend using the information in the list
- ues the 'mute' click policy so we can see a llttle bit of the bars without any legend active
- show the whole figure

In [32]:
from bokeh.models import ColumnDataSource,FactorRange
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import Legend,HoverTool
from bokeh.palettes import Category20c

output_notebook()
source = ColumnDataSource(prep_data)
colours = Category20c[14]

hours = [str(i) for i in range(24)]
p = figure(x_range = FactorRange(factors=hours),plot_width=1000, plot_height=500,x_axis_label='Hour of the day',y_axis_label= 'Frequency',
          title = 'Crime per hour')
bar ={}
items = []

for indx,i in enumerate(focuscrimes):
    bar[i] = p.vbar(x='Hour',top=i,source= source,width=0.5,color = colours[indx],muted_color=colours[indx], muted_alpha=0.02) 
    items.append((i,[bar[i]]))
    
legend = Legend(items =items,location = (0,120))

p.add_layout(legend, 'right')
p.legend.click_policy='mute'
show(p)

Loading BokehJS ...

![Movie](https://github.com/oliver-chao/homework/blob/master/assignment2.gif?raw=true "movie")